##Imports

In [1]:
import re
import pandas as pd
import json
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lucas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Extração de Dados

In [2]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df


def df_tweets_respostas(json_filename):
    df = pd.read_json(json_filename).dropna().reset_index()[['replies']]
    ds = []
    for replies in df['replies'].to_list():
        for reply in replies:
            ds.append(reply)

    reply_df = pd.DataFrame (ds).drop(columns = ['attachments'])
    return reply_df

def df_tweets_cadidatos_respostas(json_filename):
    df_tweets_cand = df_tweets_candidatos(json_filename)
    df_tweets_reps = df_tweets_respostas(json_filename)

    return (df_tweets_cand, df_tweets_reps)

# Pré-Processamento

In [3]:
def normaliza(df, coluna_alvo='text'):
  coluna_normal = coluna_alvo+"_normal"
  df[coluna_normal] = df[coluna_alvo].str.lower()
  df[coluna_normal].replace(to_replace = '[^a-zA-ZÀ-ÖØ-öø-ÿ\s]', value='', regex = True, inplace = True)
  return df
  
def tokeniza(df, coluna_alvo='text_normal'):
  stop_words = nltk.corpus.stopwords.words('portuguese')
  todos_tokens = []
  todos_textos_tokens = []
  ind = 0

  for texto_normal in df[coluna_alvo].to_list():
    tokens = nltk.word_tokenize(texto_normal, language="portuguese")
    tokens_filtrados = [token for token in tokens if token not in stop_words]
    todos_tokens.append(tokens_filtrados)
    texto_tokens = ' '.join(tokens_filtrados)
    todos_textos_tokens.append(texto_tokens)
    ind += 1

  df_tokens = pd.DataFrame({'tokens': todos_tokens, 'texto_tokens': todos_textos_tokens})
  df_concat = pd.concat([df, df_tokens], axis="columns")

  return df_concat


def normaliza_tokeniza(df, coluna_alvo = 'text'):
  df_tokens = tokeniza(normaliza(df, coluna_alvo), coluna_alvo)
  return df_tokens

  #word_regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+" #para capturarmos palavras dentro do tweet
  #df['text'] = re.sub(r'^https?:\/\/.*[\r\n]*', '', df['text'], flags=re.MULTILINE)  #para remover os links do campo de texto do tweet

# Modelagem de Tópicos

*texto em itálico*# Análise de Sentimentos

# Similaridade de Textos